# Signal Audit Visualization

This notebook visualizes the trading signal gating funnel to diagnose why trades are being rejected.

**Usage:**
1. Run diagnostics/signal_audit.py to generate signal_audit.csv
2. Run this notebook to visualize the funnel
3. Identify bottlenecks and adjust strategy parameters

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Figure size
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Load Signal Audit Data

In [ ]:
# Load CSV
data_path = Path("../user_data/diagnostics/signal_audit.csv")

if not data_path.exists():
    print(f"❌ File not found: {data_path}")
    print("Run diagnostics/signal_audit.py first to generate the audit data")
else:
    df = pd.read_csv(data_path, parse_dates=['timestamp'])
    print(f"✅ Loaded {len(df):,} records from {data_path}")
    print(f"\nDate range: {df['timestamp'].min()} to {df['timestamp'].max()}")
    print(f"\nColumns: {df.columns.tolist()}")

In [ ]:
# Preview data
df.head(10)

## 2. Overall Statistics

In [ ]:
# Calculate statistics
total_candles = len(df)
total_trades = df[['enter_long', 'enter_short']].any(axis=1).sum()
rejection_rate = ((total_candles - total_trades) / total_candles * 100) if total_candles > 0 else 0

print("="*60)
print("OVERALL STATISTICS")
print("="*60)
print(f"Total candles: {total_candles:,}")
print(f"Total trades: {total_trades}")
print(f"Overall rejection rate: {rejection_rate:.1f}%")
print(f"Trades per 1000 candles: {(total_trades / total_candles * 1000):.1f}")
print("="*60)

## 3. Gating Funnel Visualization

In [ ]:
# Calculate pass rates for each stage
stages = [
    'do_predict',
    'di_ok',
    'vol_ok',
    'regime_ok',
    'trend_strength_ok',
    'z_score_signal',
    'quantile_ok',
    'governance_ok',
]

stage_labels = [
    '1. do_predict',
    '2. DI Check',
    '3. Volume',
    '4. Regime',
    '5. Trend',
    '6. Z-Score',
    '7. Quantile',
    '8. Governance',
]

pass_counts = [df[stage].sum() for stage in stages]
pass_rates = [(count / total_candles * 100) for count in pass_counts]

# Create funnel chart
fig, ax = plt.subplots(figsize=(12, 8))

y_pos = np.arange(len(stages))
colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(stages)))

bars = ax.barh(y_pos, pass_rates, color=colors, edgecolor='black', linewidth=1.5)

# Add percentage labels
for i, (bar, rate) in enumerate(zip(bars, pass_rates)):
    width = bar.get_width()
    ax.text(
        width + 1,
        bar.get_y() + bar.get_height() / 2,
        f'{rate:.1f}% ({pass_counts[i]:,})',
        ha='left',
        va='center',
        fontsize=10,
        fontweight='bold'
    )

ax.set_yticks(y_pos)
ax.set_yticklabels(stage_labels)
ax.invert_yaxis()
ax.set_xlabel('Pass Rate (%)', fontsize=12, fontweight='bold')
ax.set_title('Signal Gating Funnel - Pass Rates', fontsize=14, fontweight='bold')
ax.set_xlim(0, 110)
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Pass Rates by Stage:")
for label, rate, count in zip(stage_labels, pass_rates, pass_counts):
    print(f"  {label:25s}: {rate:6.1f}% ({count:,} passed)")

## 4. Rejection Reasons Distribution

In [ ]:
# Count rejection reasons
rejection_counts = df['rejection_stage'].value_counts()

# Plot pie chart
fig, ax = plt.subplots(figsize=(10, 8))

colors = plt.cm.Set3(np.linspace(0, 1, len(rejection_counts)))
wedges, texts, autotexts = ax.pie(
    rejection_counts.values,
    labels=rejection_counts.index,
    autopct='%1.1f%%',
    colors=colors,
    startangle=90,
    textprops={'fontsize': 10}
)

# Make percentage text bold
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(11)

ax.set_title('Rejection Reasons Distribution', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🚫 Top Rejection Reasons:")
for reason, count in rejection_counts.items():
    pct = (count / total_candles * 100)
    print(f"  {reason:30s}: {count:6,d} ({pct:5.1f}%)")

## 5. Time Series Analysis

In [ ]:
# Resample to hourly and calculate pass rates
df_hourly = df.set_index('timestamp').resample('1H').agg({
    'do_predict': 'mean',
    'di_ok': 'mean',
    'vol_ok': 'mean',
    'regime_ok': 'mean',
    'z_score_signal': 'mean',
    'governance_ok': 'mean',
    'enter_long': 'sum',
    'enter_short': 'sum',
})

# Plot time series
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Plot 1: Pass rates over time
axes[0].plot(df_hourly.index, df_hourly['do_predict'] * 100, label='do_predict', linewidth=2)
axes[0].plot(df_hourly.index, df_hourly['di_ok'] * 100, label='DI check', linewidth=2)
axes[0].plot(df_hourly.index, df_hourly['vol_ok'] * 100, label='Volume', linewidth=2)
axes[0].plot(df_hourly.index, df_hourly['governance_ok'] * 100, label='Governance', linewidth=2)
axes[0].set_ylabel('Pass Rate (%)', fontsize=11, fontweight='bold')
axes[0].set_title('Gating Stage Pass Rates Over Time (Hourly)', fontsize=12, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(alpha=0.3)

# Plot 2: Trade entries over time
axes[1].plot(df_hourly.index, df_hourly['enter_long'], label='Long Entries', color='green', linewidth=2)
axes[1].plot(df_hourly.index, df_hourly['enter_short'], label='Short Entries', color='red', linewidth=2)
axes[1].set_ylabel('Trade Count', fontsize=11, fontweight='bold')
axes[1].set_title('Trade Entries Over Time (Hourly)', fontsize=12, fontweight='bold')
axes[1].legend(loc='best')
axes[1].grid(alpha=0.3)

# Plot 3: Overall success rate (end-to-end funnel)
df_hourly['overall_success'] = (df_hourly['enter_long'] + df_hourly['enter_short']) > 0
df_hourly['overall_success_rate'] = df_hourly['overall_success'].rolling(window=24).mean() * 100
axes[2].plot(df_hourly.index, df_hourly['overall_success_rate'], color='purple', linewidth=2)
axes[2].set_xlabel('Date', fontsize=11, fontweight='bold')
axes[2].set_ylabel('Success Rate (%)', fontsize=11, fontweight='bold')
axes[2].set_title('Overall Funnel Success Rate (24h Rolling Average)', fontsize=12, fontweight='bold')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Value Distributions

In [ ]:
# Plot distributions of key values
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# DI values
axes[0, 0].hist(df['di_value'].dropna(), bins=50, color='blue', alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('DI Value')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('DI Values Distribution')
axes[0, 0].axvline(df['di_value'].median(), color='red', linestyle='--', label=f'Median: {df["di_value"].median():.2f}')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Volume regime
axes[0, 1].hist(df['volume_regime'].dropna(), bins=50, color='green', alpha=0.7, edgecolor='black')
axes[0, 1].set_xlabel('Volume Regime')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Volume Regime Distribution')
axes[0, 1].axvline(df['volume_regime'].median(), color='red', linestyle='--', label=f'Median: {df["volume_regime"].median():.2f}')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Market regime
regime_counts = df['market_regime'].value_counts().sort_index()
axes[0, 2].bar(regime_counts.index, regime_counts.values, color='orange', alpha=0.7, edgecolor='black')
axes[0, 2].set_xlabel('Market Regime')
axes[0, 2].set_ylabel('Count')
axes[0, 2].set_title('Market Regime Distribution')
axes[0, 2].set_xticks([0, 1, 2, 3])
axes[0, 2].set_xticklabels(['Base', 'HMV', 'LMV', 'HV'])
axes[0, 2].grid(alpha=0.3, axis='y')

# Trend strength
axes[1, 0].hist(df['trend_strength'].dropna(), bins=50, color='purple', alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('Trend Strength')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Trend Strength Distribution')
axes[1, 0].axvline(0, color='red', linestyle='--', label='Zero')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Z-score
axes[1, 1].hist(df['z_score'].dropna(), bins=50, color='brown', alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Z-Score')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Z-Score Distribution')
axes[1, 1].axvline(0, color='red', linestyle='--', label='Zero')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

# Governance risk multiplier
axes[1, 2].hist(df['governance_risk_multiplier'].dropna(), bins=30, color='crimson', alpha=0.7, edgecolor='black')
axes[1, 2].set_xlabel('Risk Multiplier')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].set_title('Governance Risk Multiplier')
axes[1, 2].axvline(1.0, color='blue', linestyle='--', label='Normal (1.0)')
axes[1, 2].legend()
axes[1, 2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Recommendations

In [ ]:
print("="*60)
print("DIAGNOSTIC RECOMMENDATIONS")
print("="*60)

# Analyze each stage
recommendations = []

for i, (stage, label, rate) in enumerate(zip(stages, stage_labels, pass_rates)):
    if rate < 50.0:
        recommendations.append(f"⚠️  {label}: Very low pass rate ({rate:.1f}%)")
        
        if stage == 'do_predict':
            recommendations.append("   → Check FreqAI model training and data availability")
        elif stage == 'di_ok':
            recommendations.append("   → Consider relaxing buy_di_threshold parameter")
        elif stage == 'vol_ok':
            recommendations.append("   → Review vol_min and vol_max bounds")
        elif stage == 'regime_ok':
            recommendations.append("   → Consider allowing more market regime types")
        elif stage == 'governance_ok':
            recommendations.append("   → Check governance_decisions.jsonl for halt reasons")

if rejection_rate > 95.0:
    recommendations.append("🔴 CRITICAL: Overall rejection rate >95%!")
    recommendations.append("   → Strategy is barely trading - review all filters")
elif rejection_rate > 80.0:
    recommendations.append("🟡 WARNING: Overall rejection rate >80%")
    recommendations.append("   → Consider relaxing filters for more opportunities")
else:
    recommendations.append("✅ Overall rejection rate is acceptable")

if recommendations:
    for rec in recommendations:
        print(rec)
else:
    print("✅ No major issues detected")

print("="*60)

## 8. Export Summary Report

In [ ]:
# Create summary report
report = {
    'total_candles': total_candles,
    'total_trades': total_trades,
    'overall_rejection_rate_pct': rejection_rate,
    'stage_pass_rates': dict(zip(stage_labels, pass_rates)),
    'top_rejection_reasons': rejection_counts.to_dict(),
}

# Save to JSON
import json
report_path = Path("../user_data/diagnostics/signal_audit_report.json")
with open(report_path, 'w') as f:
    json.dump(report, f, indent=2)

print(f"✅ Summary report saved to {report_path}")